# Crawling data PTA Trunojoyo

Langkah pertama adalah import library scrapy yang nantinya akan digunakan untuk mengambil link dari halaman PTA yang akan kita ambil.

In [ ]:
import scrapy

class Url(scrapy.Spider):
    name = "url"
    start_urls = []
    
    def __init__(self):
        url = 'https://pta.trunojoyo.ac.id/c_search/byprod/7/'
        for page in range(1,3):
            self.start_urls.append(url + str(page))
        
    def parse(self, response):
        for page in range(1,6):
            for url in response.css('#content_journal > ul'):
                yield {
                    'url' : url.css('li:nth-child('+str(page)+') > div:nth-child(3) > a ::attr(href)').extract()                    
                }


Code tersebut digunakan untuk mengakses dan mengekstrak link dan mengambil link tersebut dan dijadikan 1 file dengan eksistensi .json<br>
Dengan cara input command pada Terminal : scrapy runspider /namafile.py -o namafile.json


>Mengambil data abstrak dari web PTA Trunojoyo

untuk mengambil data abstrak yang akan kita olah, kita membutuhkan library json untuk membuka dan membaca file.json yang berisi link halaman.

In [ ]:
import scrapy
import json

class Pta(scrapy.Spider):
    name = "crawling"
    file_json = open("urls.json")
    start_urls = json.loads(file_json.read())
    urls = []

    for i in range(len(start_urls)):
        b = start_urls[i]['url'][0]
        urls.append(b)
    
    def start_requests(self):
        for url in self.urls:
            yield scrapy.Request(url = url, callback = self.parse)
        
    def parse(self, response):
        # print(response.url)

        for jurnal in response.css('#content_journal > ul > li'):
            yield {
                'Judul':jurnal.css('li > div:nth-child(2) > a::text').get(),
                'Penulis':jurnal.css('li > div:nth-child(2) > div:nth-child(2) > span::text').get(),
                'Dosbing_1':jurnal.css('li > div:nth-child(2) > div:nth-child(3) > span::text').get(),
                'Dosbing_2':jurnal.css('li > div:nth-child(2) > div:nth-child(4) > span::text').get(),
                'Abstrak_indo':jurnal.css('li > div:nth-child(4) > div:nth-child(2) > p::text').get(),
                'Abstrak_english':jurnal.css('li > div:nth-child(4) > div:nth-child(4) > p::text').get()
            }


Pada fungsi parse, kita melakukan pengambilan data Abstrak Indo dengan cara mengcopy selector yang ada di bagian Abtrak Indo pada website PTA Trunojoyo.<br>
Abstrak Indo tersebut kita beri nama fitur "Contents"